In [1]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle


In [2]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [3]:
first_10k = pd.read_csv('../input/2020-digix-advertisement-ctr-prediction/train_data/train_data.csv', sep='|', nrows=10000)
first_10k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   label                        10000 non-null  int64 
 1   uid                          10000 non-null  int64 
 2   task_id                      10000 non-null  int64 
 3   adv_id                       10000 non-null  int64 
 4   creat_type_cd                10000 non-null  int64 
 5   adv_prim_id                  10000 non-null  int64 
 6   dev_id                       10000 non-null  int64 
 7   inter_type_cd                10000 non-null  int64 
 8   slot_id                      10000 non-null  int64 
 9   spread_app_id                10000 non-null  int64 
 10  tags                         10000 non-null  int64 
 11  app_first_class              10000 non-null  int64 
 12  app_second_class             10000 non-null  int64 
 13  age                          100

In [4]:
col_list = first_10k.columns.to_list()

In [5]:
for col in col_list:
    print(col)
    print(first_10k[col].unique())
    print("------")

label
[0 1]
------
uid
[1638254 1161786 1814783 ... 1354003 1293303 2095849]
------
task_id
[2112 3104 5890 ... 3475 2568 2511]
------
adv_id
[6869 3247 4183 ... 6741 1628 5099]
------
creat_type_cd
[7 2 6 3 4 8 9 5]
------
adv_prim_id
[207 183 178 138 134 126 206 175 142 104 110 193 156 177 150 112 132 131
 122 130 168 147 171 169 135 182 179 160 167 125 210 139 101 205 214 128
 209 155 102 203 143 151 129 174 173 170 107 158 127 186 114 117 133 144
 124 108 109 204 164 137 105 166 201 184 115 111 106 172 149 163 165 140
 123]
------
dev_id
[17 29 24 55 19 64 60 36 37 38 56 62 30 27 15 23 58 70 63 67 18 39 16 69
 68 52 20 33 71 13 26 47 48 65 53 54 57 22 51]
------
inter_type_cd
[5 4 3]
------
slot_id
[11 17 21 12 14 18 16 15 13 19 22 20]
------
spread_app_id
[13 86 70 33 65 57 22 34 80 50 53 58 31 11 37 78 26 56 28 44 12 46 68 79
 88 14 32 59 60 51 62 23 55 69 15 85 21 64 41 84 27 35 77 73 20 49 72 87
 38 48]
------
tags
[37 39 15 18 27 13 41 28 40 11 23 24 20 32 42 26 19 38 21 25 31

In [6]:
for col in col_list:
    print(col)
    print(first_10k[col].nunique())
    print("------")

label
2
------
uid
9924
------
task_id
1004
------
adv_id
1041
------
creat_type_cd
8
------
adv_prim_id
73
------
dev_id
39
------
inter_type_cd
3
------
slot_id
12
------
spread_app_id
50
------
tags
24
------
app_first_class
3
------
app_second_class
16
------
age
8
------
city
333
------
city_rank
4
------
device_name
77
------
device_size
56
------
career
9
------
gender
3
------
net_type
5
------
residence
33
------
his_app_size
20
------
his_on_shelf_time
4
------
app_score
2
------
emui_dev
13
------
list_time
15
------
device_price
6
------
up_life_duration
21
------
up_membership_grade
4
------
membership_life_duration
4
------
consume_purchase
9
------
communication_onlinerate
71
------
communication_avgonline_30d
14
------
indu_name
30
------
pt_d
1
------


In [7]:
temp_df = first_10k.groupby("uid").count()
temp_df2 = temp_df["creat_type_cd"]

temp_df3 = temp_df2.sort_values(ascending=False)

In [8]:
pd.set_option("display.max_colwidth", 200)

indexes = [1852341, 1738911,   1026288,   1402588,  2135240]
for i in indexes: 
    print(first_10k[first_10k.uid == i][["communication_onlinerate","communication_avgonline_30d"]])

                                         communication_onlinerate  \
240   1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23   
7148  1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23   
7149  1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23   

      communication_avgonline_30d  
240                            13  
7148                           13  
7149                           13  
                 communication_onlinerate  communication_avgonline_30d
3022  12^13^14^15^16^17^18^19^20^21^22^23                            7
8751  12^13^14^15^16^17^18^19^20^21^22^23                            7
                                 communication_onlinerate  \
84    5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23   
6483  5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23   

      communication_avgonline_30d  
84                             11  
6483                           11  
                                          communication_onlinerate  \
129 

In [9]:
merged_df = pd.DataFrame()
for col in col_list:
    print(col)
    if col != "communication_onlinerate":
        col_df = pd.read_csv('../input/2020-digix-advertisement-ctr-prediction/train_data/train_data.csv', sep='|', usecols=[col])
        downcasted = pd.to_numeric(col_df[col] , downcast='signed')
    else:
        downcasted = pd.read_csv('../input/2020-digix-advertisement-ctr-prediction/train_data/train_data.csv', sep='|', usecols=[col], dtype = {col: "category"})
    merged_df=pd.concat([merged_df,downcasted], axis =1)

label
uid
task_id
adv_id
creat_type_cd
adv_prim_id
dev_id
inter_type_cd
slot_id
spread_app_id
tags
app_first_class
app_second_class
age
city
city_rank
device_name
device_size
career
gender
net_type
residence
his_app_size
his_on_shelf_time
app_score
emui_dev
list_time
device_price
up_life_duration
up_membership_grade
membership_life_duration
consume_purchase
communication_onlinerate
communication_avgonline_30d
indu_name
pt_d


In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41907133 entries, 0 to 41907132
Data columns (total 36 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   label                        int8    
 1   uid                          int32   
 2   task_id                      int16   
 3   adv_id                       int16   
 4   creat_type_cd                int8    
 5   adv_prim_id                  int16   
 6   dev_id                       int8    
 7   inter_type_cd                int8    
 8   slot_id                      int8    
 9   spread_app_id                int8    
 10  tags                         int8    
 11  app_first_class              int8    
 12  app_second_class             int8    
 13  age                          int8    
 14  city                         int16   
 15  city_rank                    int8    
 16  device_name                  int8    
 17  device_size                  int16   
 18  career                   

In [11]:
mem_usage(merged_df)

'3237.70 MB'

In [12]:
merged_df[merged_df["age"].isna()]

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,...,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d


In [13]:
merged_df = merged_df.sample(frac = 1) 
merged_df = merged_df.reset_index(drop=True)

merged_df.head(10)

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,...,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d
0,0,1562495,2591,1076,4,112,60,3,17,37,...,8,4,20,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,12,33,4
1,0,1840849,3869,6751,7,130,30,5,21,28,...,9,3,20,-1,-1,2,7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,11,36,6
2,0,1160653,4465,2272,7,168,37,5,13,44,...,10,2,-1,-1,-1,2,7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,10,17,3
3,0,1729735,4348,3639,7,150,17,5,13,11,...,5,4,20,1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,13,36,4
4,0,2215943,3577,5884,7,207,17,5,19,13,...,17,3,20,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,13,17,1
5,0,1222917,1558,5919,6,132,60,3,17,78,...,8,5,20,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,13,47,3
6,0,2140367,1992,1016,7,156,56,5,13,58,...,9,3,20,-1,-1,2,7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,11,17,3
7,0,1955579,2642,5913,7,150,17,5,12,11,...,7,2,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,11,36,7
8,0,1768898,3994,1286,7,178,17,5,12,70,...,8,2,-1,-1,-1,2,9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,10,36,5
9,0,2215212,4100,2348,7,128,60,4,22,78,...,8,5,20,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,12,24,1


In [14]:
with open('merged_df.pkl', 'wb') as output:
    pickle.dump(merged_df, output)